In [ ]:
from __future__ import absolute_import, division, print_function

import wandb
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import random
import pprint

import tensorflow as tf
from tensorflow import concat
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, Concatenate
from tensorflow.keras.losses import CategoricalCrossentropy, Huber, MeanSquaredError
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow import reshape


from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import py_environment
from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.utils import common

from spektral.layers import XENetDenseConv
from spektral.transforms import LayerPreprocess
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data import Graph
from spektral.data.dataset import Dataset
from spektral.data.loaders import BatchLoader

from wandb.keras import WandbCallback

from collections import deque
from random import sample

In [ ]:
# RANDOM_SEED = 5
# tf.random.set_seed(RANDOM_SEED)
# env.seed(RANDOM_SEED)
# np.random.seed(RANDOM_SEED)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sweep_config = {
    "name": 'First run',
    'method':'random'
}

metric = {
    'name' : 'loss',
    'goal' : 'minimize'
}
sweep_config['metric'] = metric


parameters_dict = {
    'N_xenet': {
        'value': 1
    },
    'N_dense': {
        'distribution': 'int_uniform',
        'min': 0,
        'max': 10
    },
    'division_factor_dense': {
        'value': 1
    },
    'stack_channels': {
        'distribution': 'int_uniform',
        'min': 3,
        'max': 20
    },
    'node_channels': {
        'distribution': 'int_uniform',
        'min': 3,
        'max': 20
    },
    'edge_channels': {
        'distribution': 'int_uniform',
        'min': 3,
        'max': 20
    },
    'train_episodes': {
        'value': 220
    }    
}
sweep_config['parameters'] = parameters_dict

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'N_dense': {'distribution': 'int_uniform', 'max': 10, 'min': 0},
                'N_xenet': {'value': 1},
                'division_factor_dense': {'value': 1},
                'edge_channels': {'distribution': 'int_uniform',
                                  'max': 20,
                                  'min': 3},
                'node_channels': {'distribution': 'int_uniform',
                                  'max': 20,
                                  'min': 3},
                'stack_channels': {'distribution': 'int_uniform',
                                   'max': 20,
                                   'min': 3},
                'train_episodes': {'value': 220}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="randomSearch-project", entity="locp")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Create sweep with ID: 2srjqciz
Sweep URL: https://wandb.ai/locp/randomSearch-project/sweeps/2srjqciz


parametri
- number of train_episodes
- size of replay_memory
- update steps of main and target network
- learning_rate
- discount_factor
- MIN_REPLAY_SIZE (minimum size of the replay_memory for which we do the training)
- batch_size

## Environment

### Predefined Functions 

##### Sparse Matrix Creation

In [ ]:
def sparseAdj(D, L):
    N = L**D
    nodes = [x for x in np.ndindex(tuple(np.repeat(L,D)))]
    mul = [L**i for i in reversed(range(D))]

    A_dense = []
    for node in nodes:
        temp_buffer = []
        A_dense_row = [0]*N
        for d in range(D):
            temp=list(node)
            temp[d]=((temp[d]+1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)    

            temp=list(node)
            temp[d]=((temp[d]-1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)
      
        temp_buffer=list(np.unique(np.array(temp_buffer), axis=0))   
        for i in temp_buffer: A_dense_row[i]=1
        A_dense.append(A_dense_row)

    return sp_matrix_to_sp_tensor(np.array(A_dense))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def denseAdj(D, L):
    N = L**D
    nodes = [x for x in np.ndindex(tuple(np.repeat(L,D)))]
    mul = [L**i for i in reversed(range(D))]

    A_dense = []
    for node in nodes:
        temp_buffer = []
        A_dense_row = [0]*N
        for d in range(D):
            temp=list(node)
            temp[d]=((temp[d]+1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)    

            temp=list(node)
            temp[d]=((temp[d]-1)%L)
            temp=np.inner(temp, mul)
            temp_buffer.append(temp)
      
        temp_buffer=list(np.unique(np.array(temp_buffer), axis=0))   
        for i in temp_buffer: A_dense_row[i]=1
        A_dense.append(A_dense_row)

    return np.array(A_dense)

In [ ]:
def J_inter(denseAdj):
    N = denseAdj.shape[0]
    sparseAdj = sp_matrix_to_sp_tensor(denseAdj)
    edge=sparseAdj.indices.numpy()
    un_edge=np.array([np.sort(i) for i in edge]) 
    inter=[]
    for i in range(len(un_edge)):
        equal=True
        for j in range(i):
            if np.array_equal(un_edge[i],un_edge[j]):
                inter.append(inter[j])
                equal=False
                break
        if equal: 
            inter.append(np.random.normal(0, 1))
    
    inter_matrix = np.zeros((N,N))
    counter = 0
    for i, j in edge:
        inter_matrix[i,j] = inter[counter]
        counter += 1
    return [np.array(inter).reshape(sparseAdj.indices.shape[0],1), inter_matrix.reshape((N,N,1))]

    # 0 => interaction array
    # 1 => interaction matrix (zero padded)

In [ ]:
class SG_env(py_environment.PyEnvironment):

  def __init__(self, L, D):
    
    self.N = L**D
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=self.N-1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(self.N,1), dtype=np.int32, minimum=-1, maximum=1, name='observation')
    self.sp_AdjMat = sparseAdj(D=D, L=L)
    self.dense_AdjMat = denseAdj(D=D, L=L)
    self.interaction = J_inter(self.dense_AdjMat)[0]    # self.sp_AdjMat.indices.shape[0]=2*D*N
    self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    self._episode_ended = False

  def get_state(self):
    return self._state

  def show_N(self):
    return self.N  

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def show_dense_AdjMat(self):
    return self.dense_AdjMat

  def show_sp_AdjMat(self):
    return self.sp_AdjMat

  def show_interaction(self):
    return self.interaction

  def show_inter_matrix(self):
    return self.inter_matrix

  def __all_spins_down(self):
    return np.all(self._state==-1)    # True => All spins = -1, False otherwise

  def computeReward(self, action):
    nns = self.sp_AdjMat.indices[self.sp_AdjMat.indices[:,0]==action][:,1].numpy()
    nn_Js = np.where(self.sp_AdjMat.indices[:,0]==action)[0]
    nn_sum = 0
    for i in range(len(nns)): nn_sum += self.interaction[nn_Js[i]]*self._state[nns[i],0]
    reward = -2*nn_sum*self._state[action,0]
    return reward[0]

  def computeEnergy(self):
    edge = self.sp_AdjMat.indices.numpy()
    Nedge = len(edge)
    energy = 0
    for i in range(Nedge):
        energy -= self.interaction[i][0]*self._state[edge[i][0]][0]*self._state[edge[i][1]][0]
    return energy/2

  def _reset(self):
    self._state = np.ones(shape=(self.N,1)).astype("int32")
    #self.interaction = J_inter(self.dense_AdjMat)[0]    
    #self.inter_matrix = J_inter(self.dense_AdjMat)[1]
    self._episode_ended = False
    return ts.restart(np.array(self._state, dtype=np.int32))

  def _step(self, action):
    if self._episode_ended:
      return self.reset()

    if self.__all_spins_down():
      self._episode_ended = True
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==1):
      self._state[action,0]=-1
      rew = self.computeReward(action)
      
      if self.__all_spins_down():
          self._episode_ended = True
          return ts.termination(np.array(self._state, dtype=np.int32), reward=rew)
      else:
          return ts.transition(np.array(self._state, dtype=np.int32), reward=rew)
    
    elif (action>=0 and action<=self.N-1) and (self._state[action,0]==-1):
      return ts.transition(np.array(self._state, dtype=np.int32), reward=0)
    else:
      raise ValueError('`action` should be 0 up to N-1 - Spin Flip!')

## Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, N_graph, X, Y, A, E, **kwargs):
        self.X = X
        self.Y = Y
        self.N_graph = N_graph
        self.A = A
        self.E = E
        super().__init__(**kwargs)

    def read(self):
        mydataset = []
        for i in range(self.N_graph):
            mydataset.append(
                    Graph(x=self.X[i], a=self.A[i], e=self.E[i], y=self.Y[i])      
                    )
        return mydataset

## Agent

In [ ]:
def agent(N,         # number nodes
          D,         # number dimensions
          stack_channels=5,
          node_channels=3,
          edge_channels=3,
          division_factor_dense=4,
          p_drop=0,
          N_xenet=2,
          N_dense=2,
          activation="relu",
          regularizer=0):
  inX = Input(shape=(N,1), name='Input Nodes')
  inA = Input(shape=(N,N), name='Input Adj matrix')
  inE = Input(shape=(N,N,1), name='Input Edges')
  
  XENet_layer = XENetDenseConv(stack_channels, node_channels, edge_channels,
                     attention=True, node_activation=activation, edge_activation=activation, kernel_regularizer=l2(regularizer), name="XENet_layer")
  X, E = XENet_layer([inX, inA, inE])
  for i in range(N_xenet-1):
    X, E = XENet_layer([X, inA, E])
  
  flat_x, flat_e = Flatten(name="Nodes_encoding")(X), Flatten(name="Edges_encoding")(E)
  out = Concatenate(axis=1, name="Concatenation")([flat_x,flat_e])
  
  for i in range(N_dense):
    out = Dense(out.shape.as_list()[1]//division_factor_dense, activation=activation, kernel_regularizer=l2(regularizer))(out)
    out = Dropout(p_drop)(out)
  out = Dense(N, activation="PReLU", kernel_regularizer=l2(regularizer), name='Q-values')(out)
  
  model = Model([inX,inA,inE], out)
  model.compile(optimizer=Adam(), loss=MeanSquaredError())
  return model

## Training

In [ ]:
def train(env, replay_memory, model, target_model, done):
    #learning_rate = 0.7                                         # Learning rate
    discount_factor = 0.618
 
    MIN_REPLAY_SIZE = 32
    print("\t=> replay_memory size:", len(replay_memory))
    if len(replay_memory) < MIN_REPLAY_SIZE:
        print("\t=> EXIT\n")
        return

    batch_size = 32
    mini_batch = random.sample(replay_memory, batch_size)

    E = np.array([transition[-1] for transition in mini_batch])
    A = np.array([transition[-2] for transition in mini_batch])
    current_states = np.array([transition[0] for transition in mini_batch])
    #current_qs_list = np.array([model.predict([current_states[new_current_states[i],A[i],E[i]])[0] for i in range(batch_size)])
    current_qs_list = np.array(model.predict([current_states,A,E]))
    #current_qs_list = model.predict(current_states[1])[0]

    new_current_states = np.array([transition[3] for transition in mini_batch])
    #future_qs_list = np.array([target_model.predict(new_current_states[i],A[i],E[i])[0] for i in range(batch_size)])
    future_qs_list = np.array(target_model.predict([new_current_states,A,E]))
    #future_qs_list = target_model.predict(new_current_states).numpy()[0]

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done, dense_AdjMat, inter_matrix) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor*np.max(future_qs_list[index])
            #max_future_q = sum of reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = max_future_q

        
        X.append(observation)
        Y.append(current_qs)
    #print("\tX: ", np.array(X).shape, "\n", X)
    #print("\tY: ",np.array(Y).shape, "\n", Y)


    train_data = MyDataset(N_graph=batch_size, X=X, Y=Y, A=A, E=E)
    loader = BatchLoader(train_data, node_level=False, epochs=50, batch_size=batch_size, shuffle=False) 
    model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, verbose=2, 
              callbacks = [WandbCallback(log_evaluation=True)])


In [ ]:
def get_replay_memory(trajectory_buffer, replay_memory):
    n = len(trajectory_buffer)

    states = np.array([transition[0] for transition in trajectory_buffer])
    actions = np.array([transition[1] for transition in trajectory_buffer])
    rewards = np.array([transition[2] for transition in trajectory_buffer])
    done = np.array([transition[4] for transition in trajectory_buffer])
    inter_matrix = trajectory_buffer[0][-1]
    dense_AdjMat = trajectory_buffer[0][-2]

    cum_reward = np.cumsum(rewards)

    replay_memory.append([states[0], actions[0], cum_reward[n-1], states[n-1], done[n-1], dense_AdjMat, inter_matrix])

    return replay_memory

In [ ]:
def main(config=None):
    with wandb.init(config=config):
        config = wandb.config

        L = 3
        D = 2
        env = SG_env(L=L, D=D)
        #env = tf_py_environment.TFPyEnvironment(env)

        epsilon = 1          # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
        max_epsilon = 1      # You can't explore more than 100% of the time
        min_epsilon = 0.01   # At a minimum, we'll always explore 1% of the time
        decay = 0.01

        # 1. Initialize the Target and Main models 
        # Main Model (updated every 4 steps)
        model = agent(N=env.N, D=D, stack_channels=config.stack_channels,
                node_channels=config.node_channels, edge_channels=config.edge_channels,
                N_xenet=config.N_xenet, N_dense=config.N_dense, 
                division_factor_dense=config.division_factor_dense)

        # Target Model (updated at the end of every episode)
        target_model = agent(N=env.N, D=D, stack_channels=config.stack_channels,
                node_channels=config.node_channels, edge_channels=config.edge_channels,
                N_xenet=config.N_xenet, N_dense=config.N_dense, 
                division_factor_dense=config.division_factor_dense)
        target_model.set_weights(model.get_weights())

        energy_buffer = []
        replay_memory = []
        
        train_episodes = config.train_episodes
        for episode in range(train_episodes):
            trajectory_buffer = []                   
            total_training_rewards = 0
            steps_to_update_target_model = 0
            env.reset()
            previous_obs = np.ones(shape=(env.N,1)).astype("int32")
            done = False  
            check = np.arange(0,env.N)

            while not done: 
                observation = env.get_state()          
                print("\n\t\t\t\t++++++++++++  episode:", episode," - step:", steps_to_update_target_model, " ++++++++++++")
                #print("len(trajectory_buffer):", len(trajectory_buffer))
                #print("len(replay_memory):", len(replay_memory))

                # 2. Explore using the Epsilon Greedy Exploration Strategy
                random_number = np.random.rand()
                if random_number <= epsilon:
                    # Explore
                    action = random.choice(check)

                else:
                    # Exploit best known action
                    predicted = model([observation.reshape(1,env.N,1), env.dense_AdjMat.reshape(1,env.N,env.N), env.inter_matrix.reshape(1,env.N,env.N,1)], training=False).numpy()[0]
                    while True:
                        #check to prevent flipping the same spin twice - only once!
                        action = np.argmax(predicted)
                        if env.get_state()[action,0] == 1:
                                break;
                        predicted[action] = np.NINF

                check = np.setdiff1d(check, action)
                step_type, reward, discount, new_observation = env._step(action)
                done = env._episode_ended
                e = env.computeEnergy()
                trajectory_buffer.append([previous_obs, action, reward, new_observation, done, e, env.dense_AdjMat, env.inter_matrix])
                energy_buffer.append([episode, new_observation, env.interaction, e])

                wandb.log({
                    "Episode": energy_buffer[episode*env.N+steps_to_update_target_model][0],
                    "Step": episode*env.N+steps_to_update_target_model,
                    "New observation": wandb.Image(energy_buffer[episode*env.N+steps_to_update_target_model][1]).reshape(L,L),
                    "J interactions": energy_buffer[episode*env.N+steps_to_update_target_model][2],
                    "Energy": energy_buffer[episode*env.N+steps_to_update_target_model][3]
                })
                #print("=> observation",previous_obs)
                #print("=> new observation",new_observation)
                #print("=> action:",action)
                #print("=> reward:",reward)
                #print("=> done:",done)
                #print("=> e:",e)
                #print("=> trajectory_buffer:", trajectory_buffer)      

                if steps_to_update_target_model >= L:   
                    replay_memory = get_replay_memory(trajectory_buffer, replay_memory)
                    #print("=> replay_memory:", replay_memory, "\n")
                    trajectory_buffer = trajectory_buffer[1:]
                    #print("=> trajectory_buffer:", trajectory_buffer)


                # 3. Update the Main Network using the Bellman Equation  
                #if (steps_to_update_target_model%L==0 and steps_to_update_target_model!=0) or done:
                    #replay_memory = get_replay_memory(trajectory_buffer)
                print("\n\t\t\t\t\t      +++++ Training +++++")
                train(env, replay_memory, model, target_model, done)
    

                previous_obs = new_observation
                total_training_rewards += reward

                #print("=> total_training_rewards:", total_training_rewards, "\n")

                if done:
                    if episode >= 3:
                        #Copying main network weights to the target network weights
                        target_model.set_weights(model.get_weights())
                        
                steps_to_update_target_model += 1

            epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
        #env.close()

            

In [ ]:
wandb.agent(sweep_id, main, count=20)

wandb: Agent Starting Run: z8yhtoig with config:
wandb: 	N_dense: 7
wandb: 	N_xenet: 1
wandb: 	division_factor_dense: 1
wandb: 	edge_channels: 9
wandb: 	node_channels: 16
wandb: 	stack_channels: 17
wandb: 	train_episodes: 220
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: filippo_festa (locp). Use `wandb login --relogin` to force relogin
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.

					      +++++ Training +++++
	=> replay_memory size: 1224
1/1 - 0s - loss: 0.5156 - _timestamp: 1654001977.0000 - _runtime: 405.0000 - 41ms/epoch - 41ms/step

				++++++++++++  episode: 204  - step: 2  ++++++++++++

					      +++++ Training +++++
	=> replay_memory size: 1224
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
1/1 - 0s - loss: 0.3326 - _timestamp: 1654001977.0000 - _runtime: 405.0000 - 42ms/epoch - 42ms/step

				++++++++++++  episode: 204  - step: 3  ++++++++++++
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log val

Energy,▇▇▂▆▅▆▄▆▇█▇▆▇▆▂▅▁▇█▂▇▂▄▇▆▆▅▆▃▇▆▅▄▅▄▇▅▆▃▆
Episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▁▁▂▃▄▃▃▂▂▂▂▁▂▂▂▂▃▂▂▁▂▂▃▁▃▁▂▂▃▂▂▁▁▂▃▃▂▁
Energy,2.96697
Episode,219
Step,1979
epoch,0
loss,0.53189


wandb: Agent Starting Run: ytlp0vnh with config:
wandb: 	N_dense: 7
wandb: 	N_xenet: 1
wandb: 	division_factor_dense: 1
wandb: 	edge_channels: 8
wandb: 	node_channels: 4
wandb: 	stack_channels: 9
wandb: 	train_episodes: 220
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


				++++++++++++  episode: 204  - step: 1  ++++++++++++

					      +++++ Training +++++
	=> replay_memory size: 1224
1/1 - 0s - loss: 2.1016 - _timestamp: 1654002433.0000 - _runtime: 411.0000 - 36ms/epoch - 36ms/step

				++++++++++++  episode: 204  - step: 2  ++++++++++++
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.

					      +++++ Training +++++
	=> replay_memory size: 1224
1/1 - 0s - loss: 2.9166 - _timestamp: 1654002433.0000 - _runtime: 411.0000 - 38ms/epoch - 38ms/step

				++++++++++++  episode: 204  - step: 3  ++++++++++++
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.

					      +++++ Training +++++
	=> replay_memo

...

wandb: Agent Starting Run: xek1qasi with config:
wandb: 	N_dense: 7
wandb: 	N_xenet: 1
wandb: 	division_factor_dense: 1
wandb: 	edge_channels: 5
wandb: 	node_channels: 7
wandb: 	stack_channels: 7
wandb: 	train_episodes: 220
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



					      +++++ Training +++++
	=> replay_memory size: 1224
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
1/1 - 0s - loss: 0.6664 - _timestamp: 1654004102.0000 - _runtime: 575.0000 - 85ms/epoch - 85ms/step

				++++++++++++  episode: 204  - step: 2  ++++++++++++

					      +++++ Training +++++
	=> replay_memory size: 1224
wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.
1/1 - 0s - loss: 1.2805 - _timestamp: 1654004102.0000 - _runtime: 575.0000 - 84ms/epoch - 84ms/step

				++++++++++++  episode: 204  - step: 3  ++++++++++++

					      +++++ Training +++++
	=> replay_memory size: 1225
wandb: WARNING WandbCallback is unable to 

Energy,▅▄▄▄▁▂▂▄▄▅▅▄▅▅▄▄▆▃▅▃▃▅▃▃▃▆▆▃▆▃▃▅▆▅▃█▅▄▆▆
Episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,██▄▆▄▄▂▂▃▄▂▃▂▆▅▄▂▅▅▂▂▅▁▄▃▃▁▃▁▂▃▇▅▂▂▃▃▂▁▂
Energy,-2.71282
Episode,219
Step,1979
epoch,0
loss,1.42262


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: iki0986z with config:
wandb: 	N_dense: 10
wandb: 	N_xenet: 1
wandb: 	division_factor_dense: 1
wandb: 	edge_channels: 20
wandb: 	node_channels: 13
wandb: 	stack_channels: 11
wandb: 	train_episodes: 220
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9515997f-d5de-4c93-8ae3-fa9b47581edf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>